In [ ]:
import numpy as np
import curve
import quadrature4
import matplotlib.pyplot as plt
#%matplotlib ipympl

In [ ]:
# Figure-8
points = []
res = 30
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
for i in range(res):
    t = 2 * np.pi / res * i
    points.append(
        np.array([np.cos(t), np.sin(2*t), 0.2 * np.sin(t)])
    )
figure8 = curve.Curve(points)
curve.curvePlot(figure8,ax)

In [ ]:
# Input curve, outputs matrix
def laplacianMatrix(c, identityCoeff=1):
    N = len(c)
    L = np.zeros((N, N), dtype=np.float64)

    edgeLengths = [c.edgeLength(i) for i in range(N)]
    # Fill row
    for i in range(N):
        fi = 0.5 * np.linalg.norm(c[i+1] - c[i-1])
        denominator = edgeLengths[i-1] * edgeLengths[i] * fi
        L[i][i-1] = edgeLengths[i] / denominator
        L[i][i] = - (edgeLengths[i-1] + edgeLengths[i]) / denominator
        L[i][(i+1) % N] = edgeLengths[i-1] / denominator
    
    return L + identityCoeff * np.eye(N)
    

In [ ]:
# H1 Gradient Flow
#dE = quadrature4.dEnergy(figure8, 2, 4).list_of_points
alpha = 2
beta = 4
T = 300
deltaT = 0.004
identityCoeff = -0.1
plotFreq = 100

# Plot
graphicIndex=0
for k in range(T):
    # Compute derivative
    dE = quadrature4.dEnergy(figure8, alpha, beta).list_of_points
    # Solve Linear System
    L = laplacianMatrix(figure8, identityCoeff=identityCoeff)
    G = curve.Curve(np.linalg.solve(L, dE))
    # Evolve
    figure8 = G * deltaT + figure8
    if k % plotFreq == 0:
        fig = plt.figure()
        ax1 = fig.add_subplot(projection="3d")
        curve.curvePlot(figure8, ax1)
        ax1.set_xlim(-1.5, 1.5)
        ax1.set_ylim(-1.5, 1.5)
        ax1.set_zlim(-1.5, 1.5)
        fig.savefig(f"../figure8-H1-{graphicIndex}.png", transparent=True)
        plt.show()
        graphicIndex += 1

